In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
import tensorflow as tf
import datetime

In [ ]:
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
IMAGE_CHANNELS = 1

# IMAGE_DIRECTORY = "../Datasets/HandWritten-Nist/dataset"
IMAGE_DIRECTORY = "../Datasets/MergedMNIST/dataset"

dataset = tf.keras.preprocessing.image_dataset_from_directory(
	IMAGE_DIRECTORY,
	labels="inferred",
	label_mode="categorical", # one-hot encoding stuff
	image_size=(28, 28),
	shuffle=True,
	color_mode="grayscale",
)

# Split to train and test 80/20
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)


In [ ]:
# Get the mapping of labels to folder names
class_names = dataset.class_names

# Print the class names
print("Class names (labels mapped to folder names):", class_names)

In [ ]:
print_me = dataset.take(9)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for images, labels in print_me.take(1):
	# show the first 9 images
	for i in range(9):
		plt.subplot(330 + 1 + i)
		plt.imshow(images[i].numpy().reshape(28, 28), cmap=plt.get_cmap('gray'))
		l = np.argmax(labels[i])
		p = class_names[l]
		plt.title(f"Label={p}")
		plt.axis('off')
	plt.show()

In [ ]:
# Normalize the dataset prior to the model

def normalize(image, label):
	image = tf.cast(image, tf.float32) / 255.0
	return image, label

train_dataset = train_dataset.map(normalize)
val_dataset = val_dataset.map(normalize)

In [ ]:
# # Configuring the dataset for performance
# # Enable caching and prefetching (overlap preprocessing and model execution while training)
# AUTOTUNE = tf.data.AUTOTUNE

# # Set up a pre-processing function. It is just going to be normalizing here, but
# # that hopefully should save time later?
# def preprocess_image(image, label):
# 	image = image / 255.0
# 	return image, label

# # Cache to disk instead of memory because we got some large datasets
# dataset = dataset.cache('/tmp/dataset_cache')

# # Apply preprocessing
# # dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# # Prefetching..? Gotta be honest, this seems to help but idk how considering all
# # the images are local
# dataset = dataset.prefetch(buffer_size=2)

# train_ds = dataset.shuffle(1000)

# # train_ds = dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# # train_ds = dataset.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# # val_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

# train_dataset = train_dataset.cache('/tmp/dataset_cache')
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential()

# Hypothesis by doing tanh here stems from
# 1) Observing that LeNet while simple uses it
# 2) The initial layers are looking for general features, not specific ones
model.add(tf.keras.layers.Conv2D(
    input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
    kernel_size=5,
    filters=8,
    strides=1,
	activation=tf.keras.activations.relu,
))

# Batch normalizationhelps helps train models faster. It does this in a few
# ways. It allows us (1) to increase the learning rate (2) it helps resolve the
# vanishing gradient problem somewhat (usually done by ReLU though) (3)
# regularization which helps prevent overfitting (we could remove the dropout
# layer possibly)
# https://www.datacamp.com/tutorial/batch-normalization-tensorflow
# https://arxiv.org/abs/1502.03167
# https://www.geeksforgeeks.org/what-is-batch-normalization-in-cnn/#how-does-batch-normalization-work-in-cnn
model.add(tf.keras.layers.BatchNormalization())


model.add(tf.keras.layers.MaxPool2D(
    pool_size=(2, 2),
    strides=(2, 2)
))


# Hypothesis by doing relu here is that this layer is going to be looking for
# more abstract features and they should be an off/on situation
model.add(tf.keras.layers.Conv2D(
    kernel_size=5,
    filters=16,
    strides=1,
	activation=tf.keras.activations.relu,
))

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPool2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

model.add(tf.keras.layers.Flatten())

# LeNet uses 120 neurons, online example uses 128
# We will use 200
model.add(tf.keras.layers.Dense(
    units=200,
    activation=tf.keras.activations.relu
))

model.add(tf.keras.layers.Dropout(0.2))

# Output layer
model.add(tf.keras.layers.Dense(
    units=47,
    activation=tf.keras.activations.softmax,
    kernel_initializer=tf.keras.initializers.VarianceScaling()
))

In [ ]:
model.summary()

In [ ]:
# Compile the model

# Uses Adam. Adam works by
# - Keeping track of a history of gradients as opposed to just the current
#   gradient
# - It uses an adaptive learning rate based on that gradient history Adam is

# It also just generally efficient and works well for our use case (ChatGPT
# mentions good for computer vision tasks which is kind of what we are doing)

# We might try testing this with stochastic gradient descent instead because
# that is what we learned in class. We could use that as a storytelling aspect
# for the professor

# adam_optimizer = tf.keras.optimizers.Adam()
# model.compile(
#     optimizer=adam_optimizer,
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     metrics=['accuracy']
# )

# In past experiments, SGD has performed better and it is apparently used in
# other image networks
# https://shaoanlu.wordpress.com/2017/05/29/sgd-all-which-one-is-the-best-optimizer-dogs-vs-cats-toy-experiment/

sgd_optimizer = tf.keras.optimizers.SGD()
model.compile(
	optimizer=sgd_optimizer,
	loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
log_dir=".logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1)

training_history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
	callbacks=[early_stop]
)

print("The model has successfully trained")

In [ ]:
# Access training loss and accuracy
train_loss = training_history.history['loss']
train_accuracy = training_history.history['accuracy']

# Access validation loss and accuracy (if validation data is provided)
val_loss = training_history.history.get('val_loss')
val_accuracy = training_history.history.get('val_accuracy')

# Print statistics for each epoch
for epoch in range(len(train_loss)):
    print(f"Epoch {epoch+1}:")
    print(f"  Training Loss: {train_loss[epoch]:.4f}, Training Accuracy: {train_accuracy[epoch]:.4f}")
    if val_loss and val_accuracy:
        print(f"  Validation Loss: {val_loss[epoch]:.4f}, Validation Accuracy: {val_accuracy[epoch]:.4f}")

In [ ]:
model_name_string = "model_v10_50_epochs"

In [ ]:
import matplotlib.pyplot as plt


# Plot training and validation loss

with plt.style.context("seaborn-v0_8"):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(training_history.history['loss'], label='Training Loss')
    if 'val_loss' in training_history.history:
        plt.plot(training_history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name_string} - Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(training_history.history['accuracy'], label='Training Accuracy')
    if 'val_accuracy' in training_history.history:
        plt.plot(training_history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name_string} - Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    # plt.style.use('seaborn-v0_8-bright')
    plt.legend()

    plt.show()

In [ ]:
import pandas as pd

# Convert the history to a DataFrame and save as CSV
history_df = pd.DataFrame(training_history.history)
history_df.to_csv(f'{model_name_string}_history.csv', index=False)

In [ ]:
# Save the model itself
model.save(f'{model_name_string}.keras')

In [ ]:
# Save model weights
model.save_weights(f'{model_name_string}.weights.h5') 